### Imports

In [1]:
import time
import random
import unidecode
import string
import re

import matplotlib.pyplot as plt
import torch

torch.backends.cudnn.deterministic = True

### General Settings

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else 'cpu')

TEXT_PORTION_SIZE = 200

NUM_ITER = 5000
LEARNING_RATE = 0.005
EMBEDDING_DIM = 100
HIDDEN_DIM = 128

print('Device:', DEVICE)

Device: cuda


### Dataset

In [3]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [4]:
with open('covid19-faq.txt', 'r') as f:
    textfile = f.read()
# convert special characters
textfile = unidecode.unidecode(textfile)
# strip extra whitespaces
textfile = re.sub(' +',' ', textfile)
TEXT_LENGTH = len(textfile)
print(f'Number of characters in text: {TEXT_LENGTH}')

Number of characters in text: 84658


Divide the text into smaller portions:

In [5]:
random.seed(RANDOM_SEED)

def random_portion(textfile):
    start_index = random.randint(0, TEXT_LENGTH - TEXT_PORTION_SIZE)
    end_index = start_index + TEXT_PORTION_SIZE + 1
    return textfile[start_index:end_index]

print(random_portion(textfile))

heck with the vaccine provider.

If you're traveling away from Madison before your second dose, we encourage you to get your first dose on campus now and obtain a second dose once you reach your new lo


Define a function to convert characters into tensors of integers (type long):

In [6]:
def char_to_tensor(text):
    lst = [string.printable.index(c) for c in text]
    tensor = torch.tensor(lst).long()
    return tensor

print(char_to_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


Putting it together to make a function that draws random batches for training:

In [7]:
def draw_random_sample(textfile):    
    text_long = char_to_tensor(random_portion(textfile))
    inputs = text_long[:-1]
    targets = text_long[1:]
    return inputs, targets

In [8]:
draw_random_sample(textfile)

(tensor([ 0, 94, 24, 27, 94, 21, 10, 29, 14, 27, 94, 24, 15, 94, 26, 30, 10, 27,
         10, 23, 29, 18, 23, 14, 94, 10, 23, 13, 94, 34, 24, 30, 27, 94, 37, 10,
         13, 16, 14, 27, 94, 37, 10, 13, 16, 14, 94, 18, 23, 13, 18, 12, 10, 29,
         14, 28, 94, 63, 11, 30, 18, 21, 13, 18, 23, 16, 94, 10, 12, 12, 14, 28,
         28, 94, 16, 27, 10, 23, 29, 14, 13, 75, 63, 96, 54, 14, 14, 94, 29, 17,
         14, 94, 56, 23, 18, 31, 14, 27, 28, 18, 29, 34, 94, 43, 14, 10, 21, 29,
         17, 94, 54, 14, 27, 31, 18, 12, 14, 28, 94, 32, 14, 11, 28, 18, 29, 14,
         94, 15, 24, 27, 94, 22, 24, 27, 14, 94, 18, 23, 15, 24, 27, 22, 10, 29,
         18, 24, 23, 94, 10, 11, 24, 30, 29, 94, 26, 30, 10, 27, 10, 23, 29, 18,
         23, 14, 94, 10, 23, 13, 94, 18, 28, 24, 21, 10, 29, 18, 24, 23, 75, 96,
         96, 58, 17, 18, 21, 14, 94,  1,  4, 94, 13, 10, 34, 28, 94, 18, 28, 94,
         29, 17]),
 tensor([94, 24, 27, 94, 21, 10, 29, 14, 27, 94, 24, 15, 94, 26, 30, 10, 27, 10,
         

### Model

In [9]:
tembed = torch.nn.Embedding(num_embeddings=len(string.printable), embedding_dim=EMBEDDING_DIM)

In [10]:
out = tembed(torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [12]:
out.shape

torch.Size([10, 100])

In [14]:
class RNN(torch.nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size        
        self.embed = torch.nn.Embedding(num_embeddings=input_size, embedding_dim=embed_size)
        self.rnn = torch.nn.LSTMCell(input_size=embed_size, hidden_size=hidden_size)
        self.fc = torch.nn.Linear(hidden_size, output_size)
    
    def forward(self, character, hidden, cell_state):  # expects character as size [batch_size, 1]           
        embedded = self.embed(character) # [batch size, embedding dim] = [1, embedding dim]
        (hidden, cell_state) = self.rnn(embedded, (hidden, cell_state))
        # 1. output dim: [batch size, output_size] = [1, output_size]
        # 2. hidden dim: [batch size, hidden dim] = [1, hidden dim]
        # 3. cell dim: [batch size, hidden dim] = [1, hidden dim]
        output = self.fc(hidden) # [batch size, output_size] = [1, output_size]
        return output, hidden, cell_state
      
    def init_zero_state(self):
        init_hidden = torch.zeros(1, self.hidden_size).to(DEVICE)
        init_cell = torch.zeros(1, self.hidden_size).to(DEVICE)
        return (init_hidden, init_cell)

In [15]:
torch.manual_seed(RANDOM_SEED)
model = RNN(len(string.printable), EMBEDDING_DIM, HIDDEN_DIM, len(string.printable))
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

### Training

In [17]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    ## based on https://github.com/spro/practical-pytorch/
    ## blob/master/char-rnn-generation/char-rnn-generation.ipynb

    (hidden, cell_state) = model.init_zero_state()
    prime_input = char_to_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        inp = prime_input[p].unsqueeze(0)
        _, hidden, cell_state = model(inp.to(DEVICE), hidden, cell_state)
    inp = prime_input[-1].unsqueeze(0)
    
    for p in range(predict_len):

        outputs, hidden, cell_state = model(inp.to(DEVICE), hidden, cell_state)
        
        # Sample from the network as a multinomial distribution
        output_dist = outputs.data.view(-1).div(temperature).exp() # e^{logits / T}
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = string.printable[top_i]
        predicted += predicted_char
        inp = char_to_tensor(predicted_char)

    return predicted

In [18]:
start_time = time.time()

loss_list = []

for iteration in range(NUM_ITER):

    hidden, cell_state = model.init_zero_state()
    optimizer.zero_grad()
    
    loss = 0.
    inputs, targets = draw_random_sample(textfile)
    inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
    
    for c in range(TEXT_PORTION_SIZE):
        outputs, hidden, cell_state = model(inputs[c].unsqueeze(0), hidden, cell_state)  # inputs[c] = [1] -> inputs[c].unsqueeze(0) = [1, 1]
        loss += torch.nn.functional.cross_entropy(outputs, targets[c].view(1))

    loss /= TEXT_PORTION_SIZE
    loss.backward()
    
    ### UPDATE MODEL PARAMETERS
    optimizer.step()

    ### LOGGING
    with torch.no_grad():
        if iteration % 200 == 0:
            print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
            print(f'Iteration {iteration} | Loss {loss.item():.2f}\n\n')
            print(evaluate(model, 'Th', 200), '\n')
            print(50*'=')
            
            loss_list.append(loss.item())
#             plt.clf()
#             plt.plot(range(len(loss_list)), loss_list)
#             plt.ylabel('Loss')
#             plt.xlabel('Iteration x 1000')
#             plt.savefig('loss1.pdf')
            
# plt.clf()
# plt.ylabel('Loss')
# plt.xlabel('Iteration x 1000')
# plt.plot(range(len(loss_list)), loss_list)
# plt.show()

Time elapsed: 0.00 min
Iteration 0 | Loss 4.49


Th(\T-\`#XT&fW6BSLm1^GAqrU?w8=kH;,XATEO`+pgDgbuf=35kY3jsMJ\K<@Gh:`|ch1@%t-D^tj|80"Y
xG),qw(37-Y&plFot{6!*9tO5?E[#M4)!s@2)iRxvC8Wx:"$
,81_T)G]RxymA^8^F(r"a^nDtu_aGc( 

Time elapsed: 0.25 min
Iteration 200 | Loss 2.30


Th ated thow the fated ince gest reppeis, on the mepters) halyserstives buill th akp bice ind the the be off to their builaling if cproution will abliving of ay to stive the saline semomeilg the on to t 

Time elapsed: 0.50 min
Iteration 400 | Loss 1.71


Therivif does you be preen livate. bation and ampustal off-cand tests rocal will or appose vaccing where thes sith this the and varal their not munitins bating to be paring we the need Wacing youres as  

Time elapsed: 0.74 min
Iteration 600 | Loss 1.37


The whis is that to suby not requires a spedontaks, and to more to shonsif useral access to provided to varming be that us to campus reasome or are app Aploy not frome prole to the COVID-19 Safer for st 

Time elapsed: 